# 1. Import and Install Dependencies

In [1]:
pip uninstall tensorflow-intel

^C
Note: you may need to restart the kernel to use updated packages.


In [26]:
!pip install --user tensorflow==2.5.0 tensorflow-gpu==2.5.0 opencv-python mediapipe sklearn matplotlib

  Using cached numpy-1.19.5-cp39-cp39-win_amd64.whl (13.3 MB)
  Using cached absl_py-0.15.0-py3-none-any.whl (132 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached grpcio-1.34.1-cp39-cp39-win_amd64.whl (2.9 MB)
  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
INFO: pip is looking at multiple versions of tensorboard to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 5.6/5.6 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 970.9 kB/s eta 0:00:00
  Using cached tensorboard-2.12.0-py3-none-any.whl (5.6 MB)
  Using cached tensorboard-2.11.2-py3-none-any.whl (6.0 MB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.6.1-py3

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
jax 0.4.11 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
ml-dtypes 0.1.0 requires numpy>1.20, but you have numpy 1.19.5 which is incompatible.
tensorflow-intel 2.12.0 requires absl-py>=1.0.0, but you have absl-py 0.15.0 which is incompatible.
tensorflow-intel 2.12.0 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [28]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [29]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [30]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [31]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection

In [9]:
import cv2
import os
import mediapipe as mp
import numpy as np

# Set up directory paths
Video_DATA_PATH = os.path.join('SLT_Dataset_FYP') 
Npy_DATA_PATH = os.path.join('SLT_Dataset_NPY') 
# actions = np.array(['yes', 'no', 'help', 'please', 'thankyou', 'sorry', 'goodbye', 'hello', 'name', 'iloveyou', 'where', 
#                    'when', 'why', 'how', 'eat', 'drink', 'more', 'finished', 'bathroom', 'friend'])

actions = np.array(['yes', 'no', 'help', 'please', 'thankyou'])

no_sequences = 12
sequence_length = 198 #number of frames


In [10]:
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(Npy_DATA_PATH, action, str(sequence)))
        except:
            pass

In [11]:
Npy_DATA_PATH

'SLT_Dataset_NPY'

# 5. Collect Keypoint Values for Training and Testing

In [12]:
#printing complete numpy arrays without truncation
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [13]:
for action in actions:
    # Iterate over sequences
    for sequence in range(1,no_sequences+1):
        # Read video
        video_path = os.path.join(Video_DATA_PATH, action, str(sequence) + '.mp4')
        cap = cv2.VideoCapture(video_path)

        # Set up holistic model
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            # Iterate over frames in video
            count = 0
            keypoint_arrays = np.zeros((33,120),float) #33 keypoint with 4 coordinates each for 30 frames 4*30=120
            for frame_num in range(18,sequence_length):
                # Read frame from video
                if(frame_num % 6 == 0):
                    ret, frame = cap.read()

                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)
                    
                    # Extract and save keypoints
                    keypoints = extract_keypoints(results)
                    keypoints_num = 0
                    for i in range(0,(23*4),4): #first 23 points of pose
                        keypoint_arrays[keypoints_num,count]=keypoints[i]
                        keypoint_arrays[keypoints_num,count+1]=keypoints[i+1]
                        keypoint_arrays[keypoints_num,count+2]=keypoints[i+2]
                        keypoint_arrays[keypoints_num,count+3]=keypoints[i+3]
                        keypoints_num += 1
                    count += 4
        for i in range(23):
            npy_path = os.path.join(Npy_DATA_PATH, action, str(sequence), str(i))
            np.save(npy_path, keypoint_arrays[i])

      # Set up holistic model
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            # Iterate over frames in video
            count = 0
            keypoint_arrays = np.zeros((42,120),float)
            for frame_num in range(18,sequence_length):
                # Read frame from video
                if(frame_num % 6 == 0):
                    ret, frame = cap.read()

                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)
                    
                    # Extract and save keypoints
                    keypoints = extract_keypoints(results)
                    keypoints_num = 0
                    for i in range((33*4),len(keypoints),3):
                        keypoint_arrays[keypoints_num,count]=keypoints[i]
                        keypoint_arrays[keypoints_num,count+1]=keypoints[i+1]
                        keypoint_arrays[keypoints_num,count+2]=keypoints[i+2]
                        keypoints_num += 1
                    count += 3
        index = 0        
        for i in range(23,65):
            npy_path = os.path.join(Npy_DATA_PATH, action, str(sequence), str(i))
            np.save(npy_path, keypoint_arrays[index])
            index += 1

# 6. Preprocess Data and Create Labels and Features

In [21]:
!pip install --user --upgrade tensorflow

^C


In [22]:
!pip install --user --upgrade tensorflow tensorflow-gpu

^C


In [24]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (C:\Users\PC\anaconda3\lib\site-packages\google\protobuf\internal\__init__.py)

In [17]:
label_map = {label:num for num, label in enumerate(actions)}

In [18]:
label_map

{'yes': 0, 'no': 1, 'help': 2, 'please': 3, 'thankyou': 4}

In [13]:
sequences, labels = [], []
sequence_length = 65
for action in actions:
    for sequence in range(1,no_sequences+1):
        window = []
        for keypoint_num in range(sequence_length):
            res = np.load(os.path.join(Npy_DATA_PATH, action, str(sequence), "{}.npy".format(keypoint_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
X = np.array(sequences)
print(X.shape)

(60, 65, 120)


In [15]:
y = to_categorical(labels).astype(int)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# 7. Build and Train LSTM Neural Network

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, MaxPooling1D, Reshape, TimeDistributed, Flatten, Conv1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping


In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [132]:
#architecture 1 lstm

model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=X_train.shape[1:]))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))


In [ ]:
#architecture 2 lstm

model = Sequential()
model.add(LSTM(256,return_sequences=True, activation='relu', input_shape=X_train.shape[1:]))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(32,return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))


In [ ]:
#architecture 3 bi-lstm

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu'), input_shape=X_train.shape[1:]))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64, return_sequences=False, activation='relu')))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [111]:
#architecture 3 bi-lstm max pooling

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu'), input_shape=X_train.shape[1:]))
model.add(Bidirectional(LSTM(64, return_sequences=False, activation='relu')))
model.add(Reshape((128, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(32, return_sequences=False, activation='relu')))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(actions.shape[0], activation='softmax'))


In [35]:
# new creations

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=X_train.shape[1:]))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(actions.shape[0], activation='softmax'))



In [28]:
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=X_train.shape[1:]))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(64, activation='relu')))
model.add(Flatten())
model.add(Dense(actions.shape[0], activation='softmax'))


In [36]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [37]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_8 (Bidirection (None, 65, 256)           254976    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 32, 256)           0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 256)           0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 32, 256)           394240    
_________________________________________________________________
dropout_10 (Dropout)         (None, 32, 256)           0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 32, 128)           32896     
_________________________________________________________________
flatten_4 (Flatten)          (None, 4096)             

In [ ]:
# training model while calculating accuracy of the test data (data division : 9:3)

for f in range(2000):
    model.fit(X_train, y_train, epochs=1, callbacks=[tb_callback])
    count = 0
    res = model.predict(X_test)
    for i in range(len(res)):
        pred = actions[np.argmax(res[i])]
        act = actions[np.argmax(y_test[i])]
        if(pred ==  act):
            count += 1
    print('count : ' ,count)
    if(count == 10):
        break



2/2 [==============================] - 12s 6s/step - loss: 1.6234 - categorical_accuracy: 0.1778
count :  2
2/2 [==============================] - 3s 3s/step - loss: 1.5894 - categorical_accuracy: 0.2889
count :  3
2/2 [==============================] - 3s 3s/step - loss: 1.6126 - categorical_accuracy: 0.1778
count :  3
2/2 [==============================] - 2s 2s/step - loss: 1.5000 - categorical_accuracy: 0.3333
count :  4
2/2 [==============================] - 3s 3s/step - loss: 1.5762 - categorical_accuracy: 0.2222
count :  4
2/2 [==============================] - 3s 3s/step - loss: 1.5102 - categorical_accuracy: 0.3111
count :  3
2/2 [==============================] - 3s 3s/step - loss: 1.5190 - categorical_accuracy: 0.3556
count :  3
2/2 [==============================] - 3s 3s/step - loss: 1.5292 - categorical_accuracy: 0.2222
count :  3
2/2 [==============================] - 2s 2s/step - loss: 1.5205 - categorical_accuracy: 0.3333
count :  3
2/2 [==============================]

2/2 [==============================] - 3s 3s/step - loss: 1.0468 - categorical_accuracy: 0.5333
count :  2
2/2 [==============================] - 3s 3s/step - loss: 1.0566 - categorical_accuracy: 0.5333
count :  2
2/2 [==============================] - 4s 4s/step - loss: 1.0599 - categorical_accuracy: 0.5333
count :  2
2/2 [==============================] - 3s 3s/step - loss: 1.0529 - categorical_accuracy: 0.5333
count :  2
2/2 [==============================] - 2s 2s/step - loss: 1.0535 - categorical_accuracy: 0.5333
count :  2
2/2 [==============================] - 3s 3s/step - loss: 1.0591 - categorical_accuracy: 0.5333
count :  2
2/2 [==============================] - 3s 3s/step - loss: 1.0201 - categorical_accuracy: 0.5556
count :  3
2/2 [==============================] - 3s 3s/step - loss: 1.0477 - categorical_accuracy: 0.5333
count :  3
2/2 [==============================] - 3s 3s/step - loss: 1.0522 - categorical_accuracy: 0.4667
count :  3
2/2 [==============================] 

2/2 [==============================] - 3s 3s/step - loss: 0.9570 - categorical_accuracy: 0.6000
count :  1
2/2 [==============================] - 3s 3s/step - loss: 0.9350 - categorical_accuracy: 0.6444
count :  2
2/2 [==============================] - 2s 2s/step - loss: 0.9352 - categorical_accuracy: 0.5556
count :  1
2/2 [==============================] - 2s 2s/step - loss: 0.9769 - categorical_accuracy: 0.6222
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.8965 - categorical_accuracy: 0.6222
count :  2
2/2 [==============================] - 2s 2s/step - loss: 0.9588 - categorical_accuracy: 0.5778
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.9433 - categorical_accuracy: 0.6000
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.9932 - categorical_accuracy: 0.6000
count :  2
2/2 [==============================] - 3s 2s/step - loss: 0.9158 - categorical_accuracy: 0.6222
count :  1
2/2 [==============================] 

2/2 [==============================] - 3s 3s/step - loss: 0.5464 - categorical_accuracy: 0.7556
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.6178 - categorical_accuracy: 0.7556
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.5260 - categorical_accuracy: 0.7333
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.5406 - categorical_accuracy: 0.7556
count :  3
2/2 [==============================] - 2s 2s/step - loss: 0.5226 - categorical_accuracy: 0.7556
count :  3
2/2 [==============================] - 3s 2s/step - loss: 0.5827 - categorical_accuracy: 0.8000
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.5234 - categorical_accuracy: 0.7778
count :  2
2/2 [==============================] - 4s 4s/step - loss: 0.4248 - categorical_accuracy: 0.8000
count :  6
2/2 [==============================] - 3s 2s/step - loss: 0.6753 - categorical_accuracy: 0.7778
count :  2
2/2 [==============================] 

2/2 [==============================] - 2s 2s/step - loss: 0.3480 - categorical_accuracy: 0.8889
count :  4
2/2 [==============================] - 3s 3s/step - loss: 0.3553 - categorical_accuracy: 0.8667
count :  4
2/2 [==============================] - 3s 3s/step - loss: 0.4316 - categorical_accuracy: 0.8667
count :  4
2/2 [==============================] - 3s 3s/step - loss: 0.2679 - categorical_accuracy: 0.9556
count :  5
2/2 [==============================] - 3s 3s/step - loss: 0.3666 - categorical_accuracy: 0.8222
count :  5
2/2 [==============================] - 2s 2s/step - loss: 0.4176 - categorical_accuracy: 0.8000
count :  5
2/2 [==============================] - 3s 3s/step - loss: 0.3894 - categorical_accuracy: 0.8222
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.3566 - categorical_accuracy: 0.8444
count :  4
2/2 [==============================] - 2s 2s/step - loss: 0.3630 - categorical_accuracy: 0.8000
count :  4
2/2 [==============================] 

...

2/2 [==============================] - 4s 4s/step - loss: 0.0829 - categorical_accuracy: 0.9556
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.0327 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 4s 4s/step - loss: 0.0532 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.0374 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.0420 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 4s 4s/step - loss: 0.0985 - categorical_accuracy: 0.9556
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.0306 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 4s 4s/step - loss: 0.0495 - categorical_accuracy: 0.9778
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.0197 - categorical_accuracy: 1.0000
count :  4
2/2 [==============================] 

2/2 [==============================] - 3s 3s/step - loss: 0.1638 - categorical_accuracy: 0.9778
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.3649 - categorical_accuracy: 0.8444
count :  3
2/2 [==============================] - 2s 2s/step - loss: 0.2381 - categorical_accuracy: 0.9111
count :  3
2/2 [==============================] - 3s 2s/step - loss: 0.1566 - categorical_accuracy: 0.9333
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.1879 - categorical_accuracy: 0.9556
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.1533 - categorical_accuracy: 0.9333
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.1586 - categorical_accuracy: 0.9556
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.1686 - categorical_accuracy: 0.9111
count :  3
2/2 [==============================] - 3s 2s/step - loss: 0.1632 - categorical_accuracy: 0.9778
count :  3
2/2 [==============================] 

2/2 [==============================] - 3s 3s/step - loss: 0.0549 - categorical_accuracy: 0.9778
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.0127 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 4s 4s/step - loss: 0.0785 - categorical_accuracy: 0.9778
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.0400 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 4s 4s/step - loss: 0.0119 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 4s 4s/step - loss: 0.0472 - categorical_accuracy: 0.9778
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.0024 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 4s 3s/step - loss: 0.0083 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 4s 4s/step - loss: 0.0218 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] 

2/2 [==============================] - 3s 3s/step - loss: 0.0132 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.0155 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 2s 2s/step - loss: 0.0132 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 3s 2s/step - loss: 0.0021 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 4s 4s/step - loss: 0.0619 - categorical_accuracy: 0.9778
count :  2
2/2 [==============================] - 3s 2s/step - loss: 0.0317 - categorical_accuracy: 0.9778
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.0072 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 3s 3s/step - loss: 0.0038 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 2s 2s/step - loss: 0.0452 - categorical_accuracy: 0.9778
count :  2
2/2 [==============================] 

2/2 [==============================] - 3s 3s/step - loss: 0.0035 - categorical_accuracy: 1.0000
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.0234 - categorical_accuracy: 0.9778
count :  2
2/2 [==============================] - 3s 3s/step - loss: 0.0118 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 4s 4s/step - loss: 0.0049 - categorical_accuracy: 1.0000
count :  3
2/2 [==============================] - 5s 4s/step - loss: 0.0378 - categorical_accuracy: 0.9778
count :  3
1/2 [==============>...............] - ETA: 0s - loss: 0.0035 - categorical_accuracy: 1.0000

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
count = 0
for i in range(len(res)):
    pred = actions[np.argmax(res[i])]
    act = actions[np.argmax(y_test[i])]
    if(pred ==  act):
        count += 1

print(count)
print(len(res))

2
15


# 9. Save Weights

In [ ]:
model.save('sign_language_model.h5')

In [1]:
model.load_weights('action.h5')

NameError: name 'model' is not defined

# 10. Real Time Testing

In [33]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [32]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'prob_viz' is not defined

<Figure size 1800x1800 with 0 Axes>

In [ ]:
sequence.reverse()

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

[array([ 0.54321951,  0.47970253, -1.01231384, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.54373604,  0.48582676, -1.03276765, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.54624146,  0.48720649, -1.1176635 , ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.55150747,  0.48737022, -1.10116124, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.55633622,  0.48669183, -1.05285108, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.55960524,  0.4887619 , -1.03603768, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.56153816,  0.48901728, -1.03795612, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.56172663,  0.48231909, -1.02873945, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.56168669,  0.47917265, -1.02941298, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.56169957,  0.47050422, -1.00986612, ...,  0.        ,
         0.        ,  0.  

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

IndexError: list index out of range

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
(num_sequences,30,1662)

NameError: name 'num_sequences' is not defined

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))

array([[0.38026017, 0.577578  , 0.04216181]], dtype=float32)